# 10 years 6 leagues json

In [1]:
import pandas as pd

In [2]:
import requests
import time
import json

In [3]:
from tqdm import tqdm
import requests
import time


In [4]:
def call_api(endpoint, params=None):
    if params is None:
        params = {}

    base_url = 'https://v3.football.api-sports.io/'
    url = f"{base_url}{endpoint}"
    headers = {
        'x-rapidapi-key': '52d7e411dc82fdcbbd5e857eb2a2a8b3',  
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()  
        return response.json()  
    except requests.RequestException as e:
        print(f"API request error: {e}")
        return None


def players_data_dict(league, season, page=1, players_dict=None):
    if players_dict is None:
        players_dict = {}  

    params = {
        'league': league,
        'season': season,
        'page': page
    }

    players = call_api('players', params)

    if players:
        for player in players.get('response', []):
            player_id = player['player']['id'] 
            player_name = player['player']['name'] 

            if player_id not in players_dict:
                players_dict[player_id] = player_name  

        paging = players.get('paging', {})
        current_page = paging.get('current', 1)
        total_pages = paging.get('total', 1)

        if current_page < total_pages:
            next_page = current_page + 1
            if next_page % 2 == 1:
                time.sleep(1) 
            return players_data_dict(league, season, next_page, players_dict)

    return players_dict


def fetch_players_for_leagues_and_seasons(leagues, seasons):
    all_players_dict = {}  
    for league in leagues:
        for season in seasons:
            print(f"Fetching players for League {league}, Season {season}...")
            players_dict = players_data_dict(league, season, players_dict=all_players_dict)
            all_players_dict.update(players_dict) 
    return all_players_dict



In [6]:

league_ids = [39, 140, 78, 135, 61]  
seasons = [2024] 

all_players_data = fetch_players_for_leagues_and_seasons(league_ids, seasons)
print(f"Total unique players fetched: {len(all_players_data)}")

Fetching players for League 39, Season 2024...


Fetching players for League 140, Season 2024...
Fetching players for League 78, Season 2024...
Fetching players for League 135, Season 2024...
Fetching players for League 61, Season 2024...
Total unique players fetched: 3555


In [7]:
json_data = json.dumps(all_players_data, indent=4)  

with open('players_id_data_5league_5year.json', 'w') as json_file:
    json_file.write(json_data)

print("JSON data saved to players_id_data.json")

JSON data saved to players_id_data.json


# Image Çekme Kısmı

In [3]:

import os
import json
import requests
from tqdm import tqdm

with open('players_id_data.json', 'r') as f:
    players_data = json.load(f)  

image_folder = "data/images"
os.makedirs(image_folder, exist_ok=True) 

base_image_url = "https://media.api-sports.io/football/players/"

for player_id in tqdm(players_data.keys()):
    image_url = f"{base_image_url}{player_id}.png"
    file_path = os.path.join(image_folder, f"{player_id}.png")

    response = requests.get(image_url)

    if response.status_code == 200:
        with open(file_path, "wb") as file:
            file.write(response.content)
    else:
        print(f"Failed to download image for player ID {player_id}. Status code: {response.status_code}")


100%|██████████| 14726/14726 [49:18<00:00,  4.98it/s]  


In [5]:
import shutil

folder_path = 'data/images'
output_filename = 'data/images'

shutil.make_archive(output_filename, 'zip', folder_path)

print(f"Zipped folder saved at {output_filename}.zip")


Zipped folder saved at data/images.zip


# Detailed Json Generation

In [9]:
import json

import pandas as pd
import requests
from tqdm import tqdm
from unidecode import unidecode
import time


REQUEST_DELAY = 1 / 30  # approximately 0.133 seconds

api_key = '52d7e411dc82fdcbbd5e857eb2a2a8b3'

base_url = "https://v3.football.api-sports.io"
headers = {
    "x-apisports-key": api_key
}


players_data = {}


for player_id in tqdm(all_players_data.keys()):
    
    url = f"{base_url}/players/profiles?player={player_id}"
    response = requests.get(url, headers=headers)
    time.sleep(REQUEST_DELAY)  # Wait for the necessary time

    if response.status_code != 200:
        print(f"Error: Unable to fetch player data on id {player_id}. Status Code: {response.status_code}")
        continue

    player_info = response.json().get('response', [])[0]["player"]

    url = f"{base_url}/players/teams?player={player_id}"
    response = requests.get(url, headers=headers)
    time.sleep(REQUEST_DELAY)  # Wait for the necessary time

    if response.status_code != 200:
        print(f"Error: Unable to fetch player data on id {player_id}. Status Code: {response.status_code}")
        continue

    seasons = response.json().get('response', [])

    seasons_data = {}
    for team in seasons:
        seasons_data[team["team"]["name"]] = {"seasons": team["seasons"],
                                                "team_id": team["team"]["id"]
                                                }
    players_data[player_id] = {
        "player_info": player_info,
        "seasons_data": seasons_data
    }

with open("data/players_detailed_5league_5year.json", "w") as outfile:
    json.dump(players_data, outfile)

100%|██████████| 3555/3555 [20:35<00:00,  2.88it/s]


# Takım isimleri ve id lerinden o takımın club nation league bilgisi çıkarma

In [ ]:
df = pd.read_csv("data/team_id_to_name.csv")

In [ ]:
REQUEST_DELAY = 1 / 7.5  # approximately 0.133 seconds

api_key = '52d7e411dc82fdcbbd5e857eb2a2a8b3'

base_url = "https://v3.football.api-sports.io"
headers = {
    "x-apisports-key": api_key
}

In [ ]:

nation = []
league = []

for id in tqdm(df["team_id"]):
    url = f"{base_url}/teams?id={id}"
    response = requests.get(url, headers=headers)
    time.sleep(REQUEST_DELAY)  # Wait for the necessary time

    if response.status_code != 200:
        print(f"Error: Unable to fetch player data on page. Status Code: {response.status_code}")

    team_info = response.json().get('response', [])[0]["team"]

    nation.append(str(team_info["national"]))
    league.append(team_info["country"])

In [ ]:
df["nation"] = nation
df["league"] = league


In [ ]:
df.head()

In [ ]:
# Check columns with any None or NaN values
columns_with_na = df.columns[df.isnull().any()].tolist()
print("Columns with None/NaN values:", columns_with_na)


In [ ]:
missing_values = df[df.isnull().any(axis=1)]
print("Rows with missing values:")
print(missing_values)


In [ ]:
# Save DataFrame as CSV
df.to_csv("club_league.csv", index=False)
